In [ ]:
print ("Hello World")

Hello World


In [ ]:
!pip install coclust

In [ ]:
from coclust.evaluation.external import accuracy

#accuracy(labels, predicted_labels)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [ ]:
%tensorflow_version 1.x
import tensorflow as tf 
import numpy as np

TensorFlow 1.x selected.


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
##@title Bag of Words Encode
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from random import shuffle

#vectorizer = CountVectorizer(stop_words="english",max_features=13000)
vectorizer = CountVectorizer()

trainX = vectorizer.fit_transform(X_train)
testX = vectorizer.transform(X_train)

voc = vectorizer.vocabulary_


indices = np.arange(trainX.shape[0]) #gets the number of rows 
shuffle(indices)


print('train shape:',trainX.shape,"test shape:",testX.shape)

train shape: (20000, 11110) test shape: (20000, 11110)


In [ ]:
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import numpy as np
from tensorflow.keras.utils import to_categorical
t_2D = TSNE(n_components=2).fit_transform(trainX)
kmeans = KMeans(n_clusters=20, random_state=0).fit(t_2D)
import plotly.express as px
fig = px.scatter(x=t_2D[:,0],y=t_2D[:,1],color=kmeans.predict(t_2D))
fig.show()
to_y = to_categorical(kmeans.predict(t_2D))
true = kmeans.predict(t_2D)

In [ ]:
from sklearn.metrics.cluster import *

In [ ]:
y_test = y_train

In [ ]:
from coclust.evaluation.external import accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning:

The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.



In [ ]:
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cosine
import numpy as np
global ss

def train(cluster,h_len,batch_size,c_s,c_s2,epochs,eta,save,splits=1):

    ss=-1
    import numpy as np
    from sklearn.linear_model import LogisticRegression as LR
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score

    
    from sklearn.manifold import TSNE
    from sklearn.cluster import KMeans
    import numpy as np
    from keras.utils import to_categorical
    import plotly.express as px
    import gc

    ep = 1e-20
    temp = 1.0

    clue = cluster
    clue = clue*1.0

    x = tf.placeholder(tf.float32, shape=[None,trainX.shape[1]])
    batch = tf.placeholder(tf.int32, shape=())
    y = tf.placeholder(tf.float32, shape=[None,cluster])

    L = tf.random.uniform([h_len,trainX.shape[1]],-0.05,0.05)


    def encoder_h(x,name="classDecoder"):

            W1 = tf.Variable(tf.random.uniform([trainX.shape[1],1000],0.05,-0.05),name='W1',dtype=tf.float32)
            b1 = tf.Variable(tf.zeros([1000]),name='bias1',dtype=tf.float32)

            layer1 = tf.nn.sigmoid(tf.matmul(x,W1) + b1)

            W2 = tf.Variable(tf.random.uniform([1000,750],0.05,-0.05),name='W2')
            b2 = tf.Variable(tf.zeros([750]),name='bias2',dtype=tf.float32)

            layer2 = tf.nn.sigmoid(tf.matmul(layer1,W2) + b2)

            W3 = tf.Variable(tf.random.uniform([750,h_len],-0.05,0.05),name='WMu')
            b3 = tf.Variable(tf.zeros([h_len]),name='biasMu',dtype=tf.float32)

            W4 = tf.Variable(tf.random.uniform([750,h_len],0,0),name='WSigma')
            b4 = tf.Variable(tf.zeros([h_len]),name='biasSigma',dtype=tf.float32)

            mu = tf.matmul(layer2,W3) + b3
            log_sigma = tf.matmul(layer2,W4) + b4

            return mu,log_sigma

    def encoder_c(hs,name="C"):

        with tf.variable_scope(name):
        
            W1 = tf.Variable(tf.random.uniform([trainX.shape[1],1000],-0.05,0.05),name='W1')
            b1 = tf.Variable(tf.zeros([1000]),name='bias1',dtype=tf.float32)

            
            layer1 = tf.nn.tanh(tf.matmul(hs,W1) + b1)

            W2 = tf.Variable(tf.random.uniform([1000,cluster],-0.05,0.05),name='W2')
            b2 = tf.Variable(tf.zeros([cluster]),name='bias2',dtype=tf.float32)

            layer2 = tf.nn.tanh(tf.matmul(layer1,W2) + b2)
            return layer2


    def loss(output, y):

        xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y)    
        loss = tf.reduce_mean(xentropy)
        return loss


    def decoder(x,h,name='ClassEncoder'):

        with tf.variable_scope(name):

            R = tf.Variable(L,name='RWord')
            b = tf.Variable(tf.zeros([trainX.shape[1]]),name='biasWord')

            h_R = tf.nn.log_softmax(tf.matmul(h,R)-b)

            un_probs = tf.reduce_sum(tf.multiply(h_R,x),axis=1)

            return un_probs,R,h_R


    #h
    eps = tf.random_normal((batch,h_len), 0,1)
    mu,log_sigma = encoder_h(x)
    h = mu+tf.multiply(tf.exp(log_sigma),eps)

    #c 
    eps2 = tf.random_uniform((batch,cluster), 0,1)
    eps2 = -tf.log(-tf.log(eps2+ep)+ep)
    C = encoder_c(x)
    C2 = C+eps2
    C2 = tf.nn.softmax(C2/temp,axis=1)


    # Doing mixture 
    logits = []
    word_embedding = []
    topic = []

    for l in range(0,int(cluster)):

        un_probs,R,h_R = decoder(x,h,name="Class_"+str(l))
        
        logits.append(un_probs)
        word_embedding.append(R)
        topic.append(h_R)

    decoder_probs = tf.multiply(logits,tf.transpose(C2))
    decoder_probs = tf.reduce_sum(decoder_probs,axis=0)

    # Calcule ELBO

    kld_c = tf.reduce_sum(tf.nn.softmax(C,axis=1)*tf.log(tf.nn.softmax(C,axis=1)/(1.0/np.float(cluster))),axis=1)
    kld_h = -0.5 * tf.reduce_sum(1 - tf.square(mu) + 2 * log_sigma - tf.exp(2 * log_sigma), 1) 

    ELBO = decoder_probs - kld_c - kld_h
    mean_ELBO = -tf.reduce_mean(ELBO)

    optimizer = tf.train.AdamOptimizer(learning_rate=eta)
    optimizer2 = tf.train.AdamOptimizer(learning_rate=eta)



    clu = loss(C,y)
    step = optimizer.minimize(mean_ELBO)
    step2 = optimizer.minimize(clu)

    sess = tf.Session()
    print('Initializing...')

    init = tf.global_variables_initializer()
    sess.run(init)

    sess = tf.Session()
    #print('Initializing...')

    init = tf.global_variables_initializer()
    sess.run(init)

    for ll in range(0,10):

        begin = 0
        end = batch_size

        while end <=trainX.shape[0]:

            select = indices[begin:end]

            feed_dict = {x:trainX[select].toarray().reshape(batch_size,trainX.shape[1]),y:to_y[select],batch:batch_size}
            sess.run(step2,feed_dict=feed_dict)
            a = sess.run(clu,feed_dict=feed_dict)
            
            begin = end
            end += batch_size

        from sklearn.metrics import accuracy_score
        #feed_dict = {x:trainX.toarray(),y:to_y,batch:trainX.shape[1]}
        #np.argmax(cluster_te,axis=1) = sess.run(C,feed_dict=feed_dict)

    keep_elbo_ppr = []

    for ll in range(0,epochs):

        begin = 0
        end = batch_size

        while end <=trainX.shape[0]:

            select = indices[begin:end]

            feed_dict = {x:trainX[select].toarray().reshape(batch_size,trainX.shape[1]),batch:batch_size}
            sess.run(step,feed_dict=feed_dict)

            begin = end
            end += batch_size
                
        if ll % 1 == 0:
            
            print ("### EPOCH",ll," #####")
            
            if splits == 1:

                split = testX.shape[0]/10.0
                qtd = 0

                elbo,cluster_te = [],[]

                feed_dict = {x:testX[0:10000].toarray(),batch:10000}
                elbo,cluster_te,mu_test = sess.run([ELBO,C,mu],feed_dict=feed_dict)

                feed_dict = {x:testX[10000:].toarray(),batch:10000}
                elbo2,cluster_te2,mu_test2 = sess.run([ELBO,C,mu],feed_dict=feed_dict)

                elbo = np.concatenate((elbo,elbo2))
                cluster_te = np.concatenate((cluster_te,cluster_te2))
                mu_test = np.concatenate((mu_test,mu_test2))
                
                elbo = np.array(elbo)
                #cluster = np.array(C)

                print('print ELBO mean:',np.mean(elbo))
                print(homogeneity_score(y_test,np.argmax(cluster_te,axis=1)),completeness_score(y_test,np.argmax(cluster_te,axis=1)),v_measure_score(y_test,np.argmax(cluster_te,axis=1)))
                print(silhouette_score(mu_test,np.argmax(cluster_te,axis=1)))
                print(davies_bouldin_score(mu_test,np.argmax(cluster_te,axis=1)))
                print(calinski_harabasz_score(mu_test,np.argmax(cluster_te,axis=1)))
                print(normalized_mutual_info_score(y_test,np.argmax(cluster_te,axis=1)))
                print(accuracy(y_test,np.argmax(cluster_te,axis=1)))
                #print(silhouette_score(mu_test,np.argmax(cluster_te,axis=1)))
                #print(davies_bouldin_score(mu_test,np.argmax(cluster_te,axis=1)))
                #print(calinski_harabasz_score(mu_test,np.argmax(cluster_te,axis=1)))
                #print(int(qtd))

            else:    
                
                feed_dict = {x:testX.toarray(),batch:testX.shape[0]}
                elbo,cluster_te = sess.run([ELBO,C],feed_dict=feed_dict)
                print('print ELBO mean:',np.mean(elbo))
                print(homogeneity_score(y_test,np.argmax(cluster_te,axis=1)),completeness_score(y_test,np.argmax(cluster_te,axis=1)),v_measure_score(y_test,np.argmax(cluster_te,axis=1)))
                

    tf.reset_default_graph()
    tf.keras.backend.clear_session()
    gc.collect()




In [ ]:
cluster = [20]
h_len = [1024]
batch_size = [64,8,128,32]
c_s = 1000
c_s2 = 300
epochs = 10
eta = 0.00033
import numpy as np
for bb in batch_size:
    for cl in cluster:
        for hh in h_len:

            print("c:",cl,"h:",hh,"batch:",bb)
            #build_model(cl,hh,batch_size,c_s,c_s2,epochs,eta)
            train(cl,hh,bb,c_s,c_s2,epochs,eta,save="final_yelp"+str(hh)+"_c_"+str(cl))
            print("Done!")
    #break

c: 20 h: 1024 batch: 64


Using TensorFlow backend.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Initializing...
### EPOCH 0  #####
print ELBO mean: -67.720055
0.36168307292552204 0.36462850335624025 0.3631498158001492
0.3631498158001492
0.4183


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning:

The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.



### EPOCH 1  #####
print ELBO mean: -65.40143
0.3451571705289687 0.3494480116054638 0.34728933798646094
0.347289337986461
0.40005


...

### EPOCH 8  #####
print ELBO mean: -62.043617
0.170047705389507 0.28277425457055316 0.21237977565042157
0.21237977565042157
0.22455


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning:

The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.



### EPOCH 9  #####
print ELBO mean: -61.887264
0.1277719538733533 0.2591071962955973 0.17114715393102603
0.171147153931026
0.1832


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning:

The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.



Done!


In [ ]:
cluster = [20]
h_len = [300,512,80,200,1024]
batch_size = [64,8,128,32]
c_s = 1000
c_s2 = 300
epochs = 5
eta = 0.00033
import numpy as np
for bb in batch_size:
    for cl in cluster:
        for hh in h_len:

            print("c:",cl,"h:",hh,"batch:",bb)
            #build_model(cl,hh,batch_size,c_s,c_s2,epochs,eta)
            train(cl,hh,bb,c_s,c_s2,epochs,eta,save="final_yelp"+str(hh)+"_c_"+str(cl))
            print("Done!")
    #break

c: 20 h: 300 batch: 64


Using TensorFlow backend.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Initializing...
### EPOCH 0  #####
print ELBO mean: -66.50866
0.41914341243967923 0.42314167890830556 0.4211330559329781
0.4211330559329781
0.5147


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning:

The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.



### EPOCH 1  #####
print ELBO mean: -62.729
0.4042080170518425 0.40964409349589337 0.406907900300303
0.40690790030030294
0.5005


...

/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning:

The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.



Done!
c: 20 h: 300 batch: 8
Initializing...
### EPOCH 0  #####
print ELBO mean: -61.366974
0.13972978068719508 0.2736658958500081 0.1850008492057613
0.18500084920576132
0.2051


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning:

The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.



### EPOCH 1  #####
print ELBO mean: -60.36155
0.09115887263052702 0.289166661279592 0.1386186542542824
0.13861865425428244
0.14995


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning:

The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.



In [ ]:
cluster = [20]
h_len = [300,512,80,200,1024,150]
batch_size = [64,128,32]
c_s = 1000
c_s2 = 300
epochs = 5
eta = 0.00033
import numpy as np
for bb in batch_size:
    for cl in cluster:
        for hh in h_len:

            print("c:",cl,"h:",hh,"batch:",bb)
            #build_model(cl,hh,batch_size,c_s,c_s2,epochs,eta)
            train(cl,hh,bb,c_s,c_s2,epochs,eta,save="final_yelp"+str(hh)+"_c_"+str(cl))
            print("Done!")
    #break

c: 20 h: 300 batch: 64


Using TensorFlow backend.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Initializing...
### EPOCH 0  #####
print ELBO mean: -66.28507
0.4126073257842901 0.41634018329309846 0.41446534976937627
-0.2945998
48.597856924544814
604.0589890858938
0.41446534976937627
0.5007


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning:

The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.



### EPOCH 1  #####
print ELBO mean: -62.655056
0.40201523411460455 0.4067269446986209 0.40435736425587304
-0.2848689
59.18199501310441
580.9419137343388
0.404357364255873
0.492


...

### EPOCH 3  #####
print ELBO mean: -60.061443
0.28545301055174965 0.3389090178917255 0.3098926425795227
-0.2672371
39.456509099126144
930.0802294105008
0.30989264257952276
0.36805


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning:

The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.



### EPOCH 4  #####
print ELBO mean: -59.79437
0.28632003515183246 0.3396185502432538 0.3107001150360019
-0.26755717
37.753524967773366
912.2584883076364
0.3107001150360019
0.36905


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning:

The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.



Done!
